
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-2-uis-student/-/raw/master/imgs/Banner_IA2_1.png" style="width:1500px;heigth:100px">

In [2]:
#@title 1. MONTAR EL DRIVE  (If you are in in colab){ display-mode: "form" }
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Proyecto-IA2')
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/Proyecto-IA2


In [3]:
#@title import libs
import os
import cv2
from PIL import Image
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from PIL import Image

from tensorflow import keras
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.models import Model

In [4]:
#Import dataset image from API Kaagle
!pip install -q kaggle

In [5]:
from google.colab import files

files.upload()

KeyboardInterrupt: ignored

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ashishmotwani/tomato

In [ ]:
!unzip tomato

In [ ]:
!ls

In [ ]:
path = "/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2"
os.listdir(path)

['kaggle.json',
 'tomato.zip',
 'train',
 'valid',
 'Nodules_classification_challenge_(Student)__ (1).ipynb',
 'tomato.ipynb',
 'Copia de tomato (1).ipynb',
 'Copia de tomato.ipynb']

In [ ]:
train_path = os.path.join(path, "train")
print(os.listdir(train_path))
print("*"*100)
test_path = os.path.join(path, "valid")
print(os.listdir(test_path))

['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'healthy', 'powdery_mildew']
****************************************************************************************************
['Bacterial_spot', 'Early_blight', 'Late_blight', 'Leaf_Mold', 'Septoria_leaf_spot', 'Spider_mites Two-spotted_spider_mite', 'Target_Spot', 'Tomato_Yellow_Leaf_Curl_Virus', 'Tomato_mosaic_virus', 'healthy', 'powdery_mildew']


In [ ]:
from glob import glob
folders = glob("/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/*")
folders

['/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Bacterial_spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Early_blight',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Late_blight',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Leaf_Mold',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Septoria_leaf_spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Spider_mites Two-spotted_spider_mite',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Target_Spot',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-IA2/train/Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/.shortcut-targets-by-id/1PQzbUvOzB5hWnfzPk0QwYiRx4vW7ma3N/Proyecto-

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
image_size = [256, 256]
vgg = VGG16(input_shape = image_size + [3], weights = 'imagenet', include_top =  False)


In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation = 'softmax')(x)
model = Model(inputs = vgg.input, outputs = prediction)

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_data_gen = ImageDataGenerator(rescale = 1./255)
train_set = train_data_gen.flow_from_directory('./train/', target_size = (256,256), batch_size = 32, class_mode = 'categorical')

Found 25851 images belonging to 11 classes.


In [ ]:
test_set = test_data_gen.flow_from_directory('./valid/', target_size = (256,256), batch_size = 32, class_mode = 'categorical')

In [ ]:
mod = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)
                       

<ipython-input-48-738743a9a784>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  mod = model.fit_generator(


Epoch 1/20
  5/808 [..............................] - ETA: 4:55:31 - loss: 3.6681 - accuracy: 0.1063

KeyboardInterrupt: ignored